In [1]:
# required Python imports
import numpy as np
import funciones as f
import pandas as pd
from types import SimpleNamespace

In [2]:
# STEP 1: generate Euler angles and store in the database
database = SimpleNamespace(euler=f.explore_Euler_space(step=15))
database.euler.shape

(637, 3)

In [3]:
# Create a DataFrame
df = pd.DataFrame(database.euler, columns=['euler1', 'euler2', 'euler3'])
df

,euler1,euler2,euler3
0,0,0,0
1,0,0,15
2,0,0,30
3,0,0,45
4,0,0,60
...,...,...,...
632,90,180,30
633,90,180,45
634,90,180,60
635,90,180,75


In [4]:
# STEP 2: Generate a mesh of values defining the reference transmissión envelope
polar, azimuths = f.regular_S2_grid(n_squared=500)
T = f.Tvalues(trans=(90, 50, 20), azimuth=azimuths, polar=polar)
x, y, z = f.sph2cart(T, azimuths, polar)

In [5]:
# STEP 3: Generate 6 transmission and azimuth angles pairs for each Euler angle
angles = np.arange(0, 100, 18)  # array([ 0, 18, 36, 54, 72, 90])

def find_nearest(df, values):
    """find the index of the nearest value in
    a pandas dataframe"""
    indexes = []
    for value in values:
         indexes.append((np.abs(df - value)).idxmin())
    return indexes

# initialize variables
T_vals = []
azi_vals = []

for euler in database.euler:    
    # rotate
    x2, y2, z2 = f.rotate(coordinates=(x, y, z), euler_ang=euler)
    
    # extract XY intersection
    xy_vectors = f.extract_XY_section_fast2(x2, y2, z2)
    
    # get the indexes of specific angles
    indexes = find_nearest(xy_vectors['angles'], angles)
    
    # append values
    T_vals.append(xy_vectors.loc[indexes, ['T']].T.values.tolist()[0])
    azi_vals.append(xy_vectors.loc[indexes, ['angles']].T.values.tolist()[0])

# store values in database
database.T_values = np.array(T_vals)
database.azimuths = np.array(azi_vals) 
    

In [6]:
database

namespace(euler=array([[  0,   0,   0],
                       [  0,   0,  15],
                       [  0,   0,  30],
                       ...,
                       [ 90, 180,  60],
                       [ 90, 180,  75],
                       [ 90, 180,  90]]),
          T_values=array([[89.99953815, 86.16508606, 76.13204092, 63.747549  , 53.76041302,
                           50.00017548],
                          [87.51597218, 89.89815431, 84.89892007, 74.186601  , 61.86880925,
                           52.66877154],
                          [80.25294953, 88.19456579, 89.59503067, 83.48169875, 72.19871199,
                           60.07250368],
                          ...,
                          [79.8173089 , 67.67549837, 56.79902162, 50.48336559, 51.65118372,
                           59.85467904],
                          [87.26735975, 78.01530802, 65.68967542, 55.01669519, 50.08889513,
                           52.79582795],
                          [89.9995

In [7]:
print(database.euler.shape)
print(database.T_values.shape)
print(database.azimuths.shape)

(637, 3)
(637, 6)
(637, 6)


In [8]:
print(f'euler angles: {np.around(database.euler[0], 1)}')
print(f'T values: {np.around(database.T_values[0], 1)}')
print(f'azimuth angles: {np.around(database.azimuths[0], 1)}')

euler angles: [0 0 0]
T values: [90.  86.2 76.1 63.7 53.8 50. ]
azimuth angles: [ 0.  18.  36.1 54.1 72.1 90.2]


In [9]:
print(f'euler angles: {np.around(database.euler[500], 1)}')
print(f'T values: {np.around(database.T_values[500], 1)}')
print(f'azimuth angles: {np.around(database.azimuths[500], 1)}')

euler angles: [75 90 45]
T values: [20. 20. 20. 20. 20. 20.]
azimuth angles: [135. 135. 135. 135. 135. 135.]


If we need to store the different variables in a numpy array:

In [10]:
np.column_stack((np.around(database.T_values[0], 1),
                 np.around(database.azimuths[0], 1),
                 np.full_like(database.azimuths[0], 90)))

array([[90. ,  0. , 90. ],
       [86.2, 18. , 90. ],
       [76.1, 36.1, 90. ],
       [63.7, 54.1, 90. ],
       [53.8, 72.1, 90. ],
       [50. , 90.2, 90. ]])